In [1]:
import os
import csv
import re
import pickle
import random
import math
import dicom
import numpy as np
from tqdm import tqdm
from natsort import natsorted
from skimage import transform
from sklearn.externals import joblib
from scipy import ndimage
from matplotlib import path

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from keras.models import Sequential, Graph
from keras.layers.core import Activation, Dense, Dropout, Flatten, Merge, Reshape, Lambda
from keras.layers.core import TimeDistributedDense, TimeDistributedMerge
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ParametricSoftplus, ELU
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianDropout, GaussianNoise
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import initializations
from keras.layers.core import Layer
from keras import backend as K

# for preventing python max recursion limit error
import sys
sys.setrecursionlimit(50000)

Using Theano backend.
Using gpu device 3: Tesla K80 (CNMeM is disabled, CuDNN 4007)


In [3]:
(data_ED_train, data_ED_train_val, 
 data_ES_train, data_ES_train_val) = joblib.load('../../data_proc/trainset2_data_for_optimal_threshold_net.pkl')

In [6]:
opt_thresholds_ED = Graph()

opt_thresholds_ED.add_input(name='imgs_3d', input_shape=(1, 24, 96, 96))

opt_thresholds_ED.add_node(Convolution3D(32, 3, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'),
                           name='conv-1-1', input='imgs_3d')
opt_thresholds_ED.add_node(BatchNormalization(), name='conv-1-1-bn', input='conv-1-1')
opt_thresholds_ED.add_node(ELU(), name='conv-1-1-activ', input='conv-1-1-bn')
opt_thresholds_ED.add_node(Convolution3D(32, 3, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'),
                           name='conv-1-2', input='conv-1-1-activ')
opt_thresholds_ED.add_node(BatchNormalization(), name='conv-1-2-bn', input='conv-1-2')
opt_thresholds_ED.add_node(ELU(), name='conv-1-2-activ', input='conv-1-2-bn')
opt_thresholds_ED.add_node(MaxPooling3D(pool_size=(2, 2, 2), border_mode='valid', dim_ordering='th'),
                           name='pool-1', input='conv-1-2-activ')

opt_thresholds_ED.add_node(Convolution3D(64, 3, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'),
                           name='conv-2-1', input='pool-1')
opt_thresholds_ED.add_node(BatchNormalization(), name='conv-2-1-bn', input='conv-2-1')
opt_thresholds_ED.add_node(ELU(), name='conv-2-1-activ', input='conv-2-1-bn')
opt_thresholds_ED.add_node(Convolution3D(64, 3, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'),
                           name='conv-2-2', input='conv-2-1-activ')
opt_thresholds_ED.add_node(BatchNormalization(), name='conv-2-2-bn', input='conv-2-2')
opt_thresholds_ED.add_node(ELU(), name='conv-2-2-activ', input='conv-2-2-bn')
opt_thresholds_ED.add_node(MaxPooling3D(pool_size=(2, 2, 2), border_mode='valid', dim_ordering='th'),
                           name='pool-2', input='conv-2-2-activ')

opt_thresholds_ED.add_node(Convolution3D(128, 3, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'),
                           name='conv-3-1', input='pool-2')
opt_thresholds_ED.add_node(BatchNormalization(), name='conv-3-1-bn', input='conv-3-1')
opt_thresholds_ED.add_node(ELU(), name='conv-3-1-activ', input='conv-3-1-bn')
opt_thresholds_ED.add_node(Convolution3D(128, 3, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'),
                           name='conv-3-2', input='conv-3-1-activ')
opt_thresholds_ED.add_node(BatchNormalization(), name='conv-3-2-bn', input='conv-3-2')
opt_thresholds_ED.add_node(ELU(), name='conv-3-2-activ', input='conv-3-2-bn')
opt_thresholds_ED.add_node(Convolution3D(128, 3, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'),
                           name='conv-3-3', input='conv-3-2-activ')
opt_thresholds_ED.add_node(BatchNormalization(), name='conv-3-3-bn', input='conv-3-3')
opt_thresholds_ED.add_node(ELU(), name='conv-3-3-activ', input='conv-3-3-bn')
opt_thresholds_ED.add_node(MaxPooling3D(pool_size=(2, 2, 2), border_mode='valid', dim_ordering='th'),
                           name='pool-3', input='conv-3-3-activ')

opt_thresholds_ED.add_node(Flatten(), name='flatten', input='pool-3')
opt_thresholds_ED.add_node(Dense(1024, activation='relu'), name='fc-1', input='flatten')
opt_thresholds_ED.add_node(Dropout(0.5), name='dropout', input='fc-1')
opt_thresholds_ED.add_node(Dense(1, activation='linear'), name='fc-2', input='dropout')

opt_thresholds_ED.add_output(name='opt_intensity_threshold', input='fc-2')

opt_thresholds_ED.compile('adam', {'opt_intensity_threshold': 'mse'})

In [7]:
batch_size = 8
nb_epoch = 30

checkpointer = ModelCheckpoint(filepath='../../model_weights/weights_trainset2_thresh_optimizer_ED.hdf5',
                               verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

opt_thresholds_ED.fit(data_ED_train,
                      batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, shuffle=True,
                      validation_data=data_ED_train_val,
                      #callbacks=[checkpointer, earlystopping])
                      callbacks=[earlystopping])

Train on 475 samples, validate on 25 samples
Epoch 1/30
475/475 [==============================] - 374s - loss: 1.7918 - val_loss: 0.1452
Epoch 2/30
104/475 [=====>........................] - ETA: 290s - loss: 0.1063

KeyboardInterrupt: 